In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline

In [2]:
class dataset:
    pass
sample_data = pd.read_csv("D:\KULIAH\Semester 8\Dataset\Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv")
sample_data.to_pickle('D:\KULIAH\Semester 8\Dataset\Thursday-15-02-2018_TrafficForML_CICFlowMeter.pkl')

In [3]:
df = pd.read_pickle('D:\KULIAH\Semester 8\Dataset\Thursday-15-02-2018_TrafficForML_CICFlowMeter.pkl')
df = df[['URG Flag Cnt','SYN Flag Cnt','RST Flag Cnt','PSH Flag Cnt','Protocol',
         'Pkt Size Avg','Flow Pkts/s','FIN Flag Cnt','ECE Flag Cnt','ACK Flag Cnt','Dst Port','Label']]
df["Flow Pkts/s"] = pd.to_numeric(df["Flow Pkts/s"], errors='coerce')
df.dropna(inplace=True)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040548 entries, 0 to 1048574
Data columns (total 12 columns):
URG Flag Cnt    1040548 non-null int64
SYN Flag Cnt    1040548 non-null int64
RST Flag Cnt    1040548 non-null int64
PSH Flag Cnt    1040548 non-null int64
Protocol        1040548 non-null int64
Pkt Size Avg    1040548 non-null float64
Flow Pkts/s     1040548 non-null float64
FIN Flag Cnt    1040548 non-null int64
ECE Flag Cnt    1040548 non-null int64
ACK Flag Cnt    1040548 non-null int64
Dst Port        1040548 non-null int64
Label           1040548 non-null object
dtypes: float64(2), int64(9), object(1)
memory usage: 103.2+ MB


In [5]:
dataset.train = df.groupby('Label')
                .apply(pd.DataFrame.sample, frac=0.8)
                .reset_index(level='Label', drop=True)
dataset.test = df.drop(dataset.train.index)
dataset.label = dataset.train.Label.copy()

In [6]:
dataset.train

,URG Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,Protocol,Pkt Size Avg,Flow Pkts/s,FIN Flag Cnt,ECE Flag Cnt,ACK Flag Cnt,Dst Port,Label
74021,0,0,0,1,6,225.900000,0.441997,0,0,0,443,Benign
600616,0,0,0,0,17,78.500000,7220.216606,0,0,0,53,Benign
897552,0,0,0,0,6,164.666667,1.006194,0,0,1,443,Benign
729524,0,0,0,0,17,49.500000,4.721926,0,0,0,5355,Benign
869560,0,0,0,0,6,0.000000,5221.932115,0,0,1,80,Benign
410206,0,0,1,1,6,171.500000,5.643560,0,1,0,3389,Benign
779155,1,0,0,0,6,0.000000,222222.222222,0,0,1,52170,Benign
970174,0,0,0,0,6,7.750000,8639.308855,0,0,1,49807,Benign
127130,1,0,0,0,6,0.000000,62500.000000,0,0,1,52155,Benign
449464,0,0,0,0,17,62.500000,150.818189,0,0,0,53,Benign


In [7]:
dataset.label.unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
      dtype=object)

In [8]:
d1 = dataset.train.replace('Benign', 0)

In [9]:
d2 = d1.replace('DoS attacks-GoldenEye', 1)

In [10]:
d3 = d2.replace('DoS attacks-Slowloris', 1)

In [11]:
d6_label = d3.Label.copy()

In [12]:
d6_label.unique()

array([0, 1], dtype=int64)

In [13]:
d6_label.value_counts()

0    790440
1     41998
Name: Label, dtype: int64

In [14]:
dataset.test_label = dataset.test.Label.copy() #ra kanggo

In [16]:
dataset.test_label.unique() #ra kanggo

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
      dtype=object)

In [15]:
a1_label = dataset.test.Label.copy()

In [16]:
a1_label.unique()

array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
      dtype=object)

In [17]:
a1 = dataset.test.replace('Benign', 0)

In [18]:
a2 = a1.replace('DoS attacks-GoldenEye', 1)

In [19]:
a3 = a2.replace('DoS attacks-Slowloris', 1)

In [20]:
a5_label = a3.Label.copy()

In [21]:
a5_label.unique()

array([0, 1], dtype=int64)

In [22]:
a5_label.value_counts()

0    197610
1     10500
Name: Label, dtype: int64

In [23]:
category_variables = ["Protocol"]

for cv in category_variables:
    d3[cv] = d3[cv].astype("category")
    a3[cv] = a3[cv].astype("category")
    
    print("Length of Categories for {} are {}".format(cv , len(d3[cv].cat.categories)))
    print("Categories for {} are {} \n".format(cv ,d3[cv].cat.categories))

Length of Categories for Protocol are 3
Categories for Protocol are Int64Index([0, 6, 17], dtype='int64') 



In [24]:
dummy_variables_2labels = category_variables
    
class preprocessing:
    train_labels = pd.get_dummies(d3, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)
    test_labels = pd.get_dummies(a3, columns = dummy_variables_2labels, prefix=dummy_variables_2labels)

In [27]:
preprocessing.test_labels.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208110 entries, 0 to 1048571
Data columns (total 14 columns):
URG Flag Cnt    208110 non-null int64
SYN Flag Cnt    208110 non-null int64
RST Flag Cnt    208110 non-null int64
PSH Flag Cnt    208110 non-null int64
Pkt Size Avg    208110 non-null float64
Flow Pkts/s     208110 non-null float64
FIN Flag Cnt    208110 non-null int64
ECE Flag Cnt    208110 non-null int64
ACK Flag Cnt    208110 non-null int64
Dst Port        208110 non-null int64
Label           208110 non-null int64
Protocol_0      208110 non-null uint8
Protocol_6      208110 non-null uint8
Protocol_17     208110 non-null uint8
dtypes: float64(2), int64(9), uint8(3)
memory usage: 19.6 MB


In [29]:
d3.head()

,URG Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,Protocol,Pkt Size Avg,Flow Pkts/s,FIN Flag Cnt,ECE Flag Cnt,ACK Flag Cnt,Dst Port,Label
74021,0,0,0,1,6,225.900000,0.441997,0,0,0,443,0
600616,0,0,0,0,17,78.500000,7220.216606,0,0,0,53,0
897552,0,0,0,0,6,164.666667,1.006194,0,0,1,443,0
729524,0,0,0,0,17,49.500000,4.721926,0,0,0,5355,0
869560,0,0,0,0,6,0.000000,5221.932115,0,0,1,80,0


In [28]:
preprocessing.test_labels.head()

,URG Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,Pkt Size Avg,Flow Pkts/s,FIN Flag Cnt,ECE Flag Cnt,ACK Flag Cnt,Dst Port,Label,Protocol_0,Protocol_6,Protocol_17
0,0,0,0,0,0.000000,0.026633,0,0,0,0,0,1,0,0
22,0,0,0,1,240.045455,0.740696,0,0,0,22,0,0,1,0
26,0,0,0,0,0.000000,0.026633,0,0,0,0,0,1,0,0
31,0,0,0,1,181.160000,0.944924,0,0,0,22,0,0,1,0
32,0,0,0,0,0.000000,0.026633,0,0,0,0,0,1,0,0


In [47]:
preprocessing.train_labels.to_csv("D:\KULIAH\Semester 8\Dataset\preprocessed_train_4_new.csv")
preprocessing.test_labels.to_csv("D:\KULIAH\Semester 8\Dataset\preprocessed_test_4_new.csv")

In [ ]:
preprocessing.train_labels.head()